In [1]:
import sys
import pickle 

sys.path.append(sys.path[0] + "/..")  # Adds higher directory to python modules path.

import numpy as np
from Functions import renormalize, scomplex, plotting
import matplotlib.pyplot as plt
import networkx as nx

import xgi
from itertools import combinations, product
from tqdm import tqdm
import seaborn as sns
plt.rcParams["text.usetex"] = False


colors = ["#003F5C","#2F4B7C","#665191","#A05195","#D45087","#F95D6A","#FF7C43","#FFA600"]
colors_sequential = colors + colors + colors + colors 
colors = ["#02405c", "#fea600", "#a20655", "#5e96c5", "#4b3596", "#fa6ca9", "#8d6cf6"]
colors_curves = colors+ colors+ colors+ colors


### Functions

In [2]:
def adjacency_of_order_hg(sc,k,l):
    # sc: hypergraph object
    # k: order of the diffusing simplices
    # l: order of the interaction simplices

    keys = ["nodes", "edges", "faces", "tetrahedra", "4-simplices"]
    nk = sc[f"n{k}"]
    adj = np.zeros((nk,nk),dtype = int)

    if l < k: 
       
        diff_units = sc[keys[k]]
        
        if l == 0:
            for i in range(nk):
                for j in range(i+1,nk):
                    intersection = (set(diff_units[i,:]) & set(diff_units[j,:]))
                    adj[i,j] = 2*len(intersection)
                    
        else: 
            edge_dict, face_dict, tet_dict = scomplex.make_dict(sc)
            dicts = [{(i,):i for i in range(sc["n0"])},edge_dict,face_dict,tet_dict]
            for i in range(nk):
                for j in range(i+1,nk):
                    intersection = (set(diff_units[i,:]) & set(diff_units[j,:]))
                    combs = list(combinations(intersection, l+1))
                    for c in combs:
                        if c in dicts[l]:
                            adj[i,j] += 2
                

    elif l > k:
        edge_dict, face_dict, tet_dict = scomplex.make_dict(sc)
        dicts = [{(i,):i for i in range(sc["n0"])},edge_dict,face_dict,tet_dict]
        for i,simp in enumerate(sc[keys[l]]):
            combs = list(combinations(simp, k+1))
            ncombs = len(combs)
            combs_present = []
            for c in range(ncombs):
                if combs[c] in dicts[k]:
                    combs_present.append(dicts[k][combs[c]])

            for c1 in combs_present:
                for c2 in combs_present:
                    if c2 != c1:
                        adj[c1,c2] += 1

    return (adj + adj.T)//2

def XO_laplacian_hg(sc,k,l):
    A = adjacency_of_order_hg(sc,k,l)
    K = np.sum(A, 0)
    L = np.diag(K) - A
    return L

In [3]:

def induce_simplices_hg(sc, mapnodes):
    # Finds induced simplices in the simplicial complex after its nodes are coarse grained 
    # INPUTS
    # sc: simplicial complex object
    # mapnodes: mapping from each node in sc to the label of its signature
  
    # OUTPUTS
    # new_sc: coarse grained simplicial complex object

    keys = ["edges", "faces", "tetrahedra", "4-simplices"]
    new_sc = {
        "nodes": np.sort(np.unique(mapnodes)),
    }
    new_sc["n0"] = len(new_sc["nodes"])
    new_sc["nodes"] = np.reshape(new_sc["nodes"], (new_sc["n0"], 1))
    for key in keys:
        new_sc[key] = []

    # Connect supernodes with hyperedges
    for order, key in enumerate(keys):
        for i in range(sc[f"n{order+1}"]):
            nodes = mapnodes[sc[key][i, :]]
            un = np.unique(nodes)
            lun = len(un)
            if lun > 1:
                new_sc[keys[lun-2]].append(un)

    # Remove duplicate hyperedges
    for order, key in enumerate(keys):
        if len(new_sc[key]) != 0:
            new_sc[key] = np.unique(
                np.sort(np.array(new_sc[key], dtype=int), axis=1), axis=0
            )
            new_sc[f"n{order+1}"] = new_sc[key].shape[0]
        else:
            new_sc[key] = np.zeros((0, order+2), dtype=int)
            new_sc[f"n{order+1}"] = 0

    return new_sc

def renormalize_single_step_hg(sc,tau, diff_order =0, int_order = 1, D = None, U = None, VERBOSE = True):
    # Performs a single step of higher-order Laplacian renormalization 
    # INPUTS
    # sc: simplicial complex object
    # tau: diffusion time
    # diff_order: order of the diffusing simplices
    # int_order: order of the interaction simplices
    # D: the list of Laplacian eigenvlaues, if None computes them from scratch
    # U: the list of Laplacian eigenvectors, if None computes them from scratch 
    # VERBOSE: if True print the number of nodes after the coarse-graining

    # OUTPUTS
    # new_sc: renormalized simplicial complex
    # mapnodes: array associating to each node in sc the node in new_sc it is mapped to
    # clusters: cluster label of each simplex of order diff_order
  

    if (D is None) or (U is None):
        L = XO_laplacian_hg(sc, diff_order, int_order)
        D,U = np.linalg.eigh(L)

    rho  = np.abs(U@np.diag(np.exp(-tau*D))@U.T)

    Gv = nx.Graph()
    Gv.add_nodes_from([i for i in range(sc[f"n{diff_order}"])])
    for i in range(sc[f"n{diff_order}"]):
        for j in range(i+1,sc[f"n{diff_order}"]):
            if rho[i,j] >= min(rho[i,i],rho[j,j]):
                Gv.add_edge(i,j)

        
    idx_components = {u:i for i,node_set in enumerate(nx.connected_components(Gv)) for u in node_set}
    clusters = [idx_components[u] for u in Gv.nodes]

    mapnodes,__ = renormalize.coarse_grain(sc,diff_order,clusters,np.max(clusters)+1)
    new_sc = induce_simplices_hg(sc, mapnodes)

    if VERBOSE:
        print(new_sc["n0"])

        
    return new_sc, mapnodes, clusters  


In [4]:
def hg_to_sc(H):
    H.cleanup()

    sc = {}
    sc["nodes"] = np.sort(np.array([H.nodes]).T,0)
    sc["n0"] = sc["nodes"].shape[0]
    keys = ["edges","faces","tetrahedra","4-simplices"]

    for k in keys:
        sc[k] = []
    for e in H.edges.members():
        if len(e) <= 5:
            sc[keys[len(e)-2]].append(list(e))

    for i,k in enumerate(keys):
        if len(sc[k]) == 0:
            sc[k] = np.zeros((0,i+2))
        else:
            sc[k] = np.unique(np.sort(np.array(sc[k]),1),axis =0)
        
        sc[f"n{i+1}"] = sc[k].shape[0] 
    return sc

In [5]:
def laplacians(dim = 2):
    if dim == 4:
        laps = ["01","02","03","04","10","12","13","14","20","21","23","24","30","31","32","34","40","41","42","43"]
    elif dim == 3:
        laps = ["01","02","03","10","12","13","20","21","23","30","31","32"]
    elif dim == 2:
        laps = ["01","02","10","12","20","21"]
    elif dim == 1:
        laps = ["01","10"]
    return laps

# XGI Hypergraphs test

In [6]:
#Load the data
#xgi.load_xgi_data()

#select hypergraph with max HOI = 5 nodes
names_hg = ['congress-bills', 'contact-high-school', 'contact-primary-school', 'diseasome', 'disgenenet','email-enron', 'email-eu', 'hospital-lyon', 'house-bills', 'house-committees', 'hypertext-conference', 'invs13', 'invs15','kaggle-whats-cooking', 'malawi-village','ndc-classes', 'ndc-substances', 'science-gallery', 'senate-bills', 'senate-committees','sfhh-conference']
true_names_hg = ['US Congress Bills', 'High school contacts', 'Primary school contact', 'Diseasome', 'Disgenenet', 'Enron email', 'Eu email', 'Hospital contacts', 'House bills', 'House committees', 'Hypertext contacts', 'InVS13', 'InVS15', 'Kaggle Competition', 'Malawi contacts', 'Classes of NDC', 'Substances of NDC', 'Science Gallery', 'Senate bills', 'Senate committees', 'SFHH conference']

Save/Import hypergraphs

In [7]:
#xgi datasets
save = False
if save == True:
    #save the hypergraphs
    with open("../Experiments_results/xgi/xgi_sc.pkl", "wb") as f:
        pickle.dump(scs, f)
else:
    #import the hypergraphs in our format
    with open("../Experiments_results/xgi/xgi_sc.pkl", "rb") as f:
        scs = pickle.load(f)

<h1>Compute Laplacians an Sp Heats</h1>

In [8]:
N_data = len(names_hg)
print('Number of datasets =', N_data)

Number of datasets = 21


In [9]:
SP_HEATS = {}
for name in tqdm(names_hg):
    #select dataset
    sc = scs[name]
    smax = 0
    for i in range(5):
        if sc[f'n{i}'] != 0:
            smax +=1
    # Define strings which specify the cross-order Laplacians to consider
    laplacians_types = laplacians(smax - 1)

    sparse = False
    num_eigs = 500

    C_curves = {l : [] for l in laplacians_types}
    taumin = -3 # Heat curve starts from 10**taumin
    taumax = 5 # Heat curve ends at 10**taumax
    ntau = 200 # Number of taus to consider in the interval
    for idl,l in tqdm(enumerate(laplacians_types)):    
        ## Configuration model
        L = XO_laplacian_hg(sc, k=int(l[0]), l=int(l[1]))
            
        if sparse:
            D,__ = scipy.sparse.linalg.eigsh(L.asfptype(),k = num_eigs, which = "SM")
            D = np.append(D,1000000*np.ones(L.shape[0]-num_eigs),axis=0)
        else:
            D,__ = np.linalg.eigh(L)
            D = np.abs(D)
        entropic_susceptibility,tau_space,__  = renormalize.compute_entropic_C(D,taumin,taumax,ntau)
        C_curves[l] = entropic_susceptibility
    SP_HEATS[name] = C_curves


  0%|          | 0/21 [00:00<?, ?it/s]

01


/opt/homebrew/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/Users/martamorandini/Desktop/projects/Simplicial_Renormalization/higher_order_LRG/HOLR/Experiments_setups/../Functions/renormalize.py:28: RuntimeWarning: overflow encountered in exp
  mu[i] = 1 / np.sum(np.exp(-tau * (D - D[i])))


02
03
04
10


  0%|          | 0/21 [01:07<?, ?it/s]


KeyboardInterrupt: 

Save specific heats for the hypergraphs

In [11]:
with open('../Experiments_results/xgi/xgi_spheats.pkl', "wb") as f:
    pickle.dump(SP_HEATS, f)

Import specific heats for null model

In [13]:
with open('../Experiments_results/xgi/xgi_spheats_cm.pkl', "rb") as f:
    SP_HEATS_CM = pickle.load(f)

In [ ]:
for name in tqdm(['congress-bills', 'email-eu']):
    #select dataset
    sc = scs[name]
    smax = 0
    for i in range(5):
        if sc[f'n{i}'] != 0:
            smax +=1
    
    NULL_MODEL = True
    nrep = 3
    if NULL_MODEL:
        G = nx.from_edgelist(sc["edges"])

    sparse = False
    num_eigs = 500

    # Define strings which specify the cross-order Laplacians to consider
    laplacians_types = laplacians(smax - 1)

    C_curves = {l : [] for l in laplacians_types}
    taumin = -3 # Heat curve starts from 10**taumin
    taumax = 5 # Heat curve ends at 10**taumax
    ntau = 200 # Number of taus to consider in the interval

    if NULL_MODEL:
        for idl,l in tqdm(enumerate(laplacians_types)): 
            A = adjacency_of_order_hg(sc, k=int(l[0]), l=int(l[1]))
            for n in tqdm(range(nrep)):
                # # Configuration model
                Gcm = nx.Graph(A)
                Gcm = nx.configuration_model([val for (__, val) in Gcm.degree()])
                L = XO_laplacian_hg(sc, k=int(l[0]), l=int(l[1]))
                
                if sparse:
                    D,__ = scipy.sparse.linalg.eigsh(L.asfptype(),k = num_eigs, which = "SM")
                    D = np.append(D,1000000*np.ones(L.shape[0]-num_eigs),axis=0)
                else:
                    D,__ = np.linalg.eigh(L)
                    D = np.abs(D)
                entropic_susceptibility,tau_space,__  = renormalize.compute_entropic_C(D,taumin,taumax,ntau)
                C_curves[l].append(entropic_susceptibility)


    else:
        for idl,l in tqdm(enumerate(laplacians_types)):    
            ## Configuration model
            L = XO_laplacian_hg(sc, k=int(l[0]), l=int(l[1]))
            
            if sparse:
                D,__ = scipy.sparse.linalg.eigsh(L.asfptype(),k = num_eigs, which = "SM")
                D = np.append(D,1000000*np.ones(L.shape[0]-num_eigs),axis=0)
            else:
                D,__ = np.linalg.eigh(L)
                D = np.abs(D)
            entropic_susceptibility,tau_space,__  = renormalize.compute_entropic_C(D,taumin,taumax,ntau)
            C_curves[l] = entropic_susceptibility
    SP_HEATS_CM[name] = C_curves


Save null model

In [ ]:
with open('../Experiments_results/xgi/xgi_spheats_cm.pkl', "wb") as f:
    pickle.dump(SP_HEAT_CM, f)